In [30]:
import pandas as pd
import os
import ast
import json
import requests
from typing import Dict, List

In [2]:
cache_dir = "/home/ec2-user/SageMaker"
os.environ['HF_HOME'] = cache_dir

In [21]:
df = pd.read_csv('preprocessed_data/names/named_test_set_proc_final.csv')
df['combined_entities'] = df['combined_entities'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
named_df = df[df['combined_entities'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
entities = []
for entity_list in named_df['combined_entities']:
    entities.extend(entity_list)

In [ ]:
list(set(entities))

In [33]:
named_df

,Unnamed: 0,combined_entities
2,2,"[Condi, Prime Minister Koizumi, Hu Jintao, Vla..."
3,3,"[Condi, Hizballah, Peter, Kofi Annan, Peter Wa..."
4,4,[General Casey]
5,5,"[Mark, Hutcheson, Ken]"
6,6,"[General Casey, Senator Warner, Baker, Jim, Se..."
...,...,...
302,302,"[General Dempsey, Prime Minister, General Case..."
303,303,"[Mr., NP Don Gonyea, Usama bin Laden, Bret]"
304,304,[Condi]
305,305,"[Bill Plante, Putin]"


In [34]:
import requests
import json

search_query = 'McCain'
number_of_results = 1
headers = {
  'Authorization': '',
  'User-Agent': 'thesis'
}

url = 'https://api.wikimedia.org/core/v1/wikipedia/en/search/page'
parameters = {'q': search_query, 'limit': number_of_results}
response = requests.get(url, headers=headers, params=parameters)
response = json.loads(response.text)
print(response['pages'])

[{'id': 16803137, 'key': 'Meghan_McCain', 'title': 'Meghan McCain', 'excerpt': '&quot;Citizen <span class="searchmatch">McCain</span> with Meghan <span class="searchmatch">McCain</span> | Podcast on Podbay&quot;. Podbay. Retrieved June 25, 2025. https://unt-meghan-<span class="searchmatch">mccain</span>.simplecast.com/ &quot;Meghan <span class="searchmatch">McCain</span> Uses Usha', 'matched_title': None, 'anchor': None, 'description': 'American television personality (born 1984)', 'thumbnail': {'mimetype': 'image/jpeg', 'width': 60, 'height': 84, 'duration': None, 'url': '//upload.wikimedia.org/wikipedia/commons/thumb/0/07/Meghan_McCain_2018.jpg/60px-Meghan_McCain_2018.jpg'}}]


In [25]:
def get_wikipedia_intro_with_category(title):
    headers = {
        'Authorization': '',
        'User-Agent': 'thesis'
    }

    endpoint = 'https://en.wikipedia.org/w/api.php'
    params = {
        'format': 'json',
        'action': 'query',
        'prop': 'extracts|pageprops',
        'exintro': 1,
        'explaintext': 1,
        'redirects': 1,
        'titles': title,
    }

    response = requests.get(endpoint, params=params, headers=headers, timeout=10)

    if not response.ok:
        return {
            'information': [],
            'metadata': 'respond error',
        }

    data = response.json()
    page = next(iter(data['query']['pages'].values()))
    extract = page.get('extract', '')
    pageprop = [cat for cat in page.get('pageprops', [])]
    is_disambiguation = any('disambiguation' in cat.lower() for cat in pageprop)
    metadata = 'mutliple entries'

    idx = 2 if is_disambiguation else 0
    info_list = [line for line in extract.split('\n')[idx:] if line]

    if not info_list:
            return {'information': [], 'metadata': 'no info'}

    if is_disambiguation:
            metadata = 'multiple entries'
    else:
            info_list = info_list[:1]
            metadata = 'one entry'
    return {'information': info_list, 'metadata': metadata}

result = get_wikipedia_intro_with_category('McCain')
print(result)

{'information': ['McCain (surname), a surname (includes a list of persons and characters)'], 'metadata': 'multiple entries'}


In [31]:
def get_wikipedia_intro_with_category(title: str) -> Dict[str, List]:
    """
    Fetch the introductory paragraph(s) from a Wikipedia page for a given title.

    Args:
        title (str): The Wikipedia page title to query.

    Returns:
        Dict[str, Any]: A dictionary containing:
            - 'information': List[str] of extracted intro text lines.
            - 'metadata': str indicating status ('one entry', 'multiple entries',
                          'no info', or 'respond error').
    """

    headers = {'Authorization': '', 'User-Agent': 'thesis'}
    endpoint = 'https://en.wikipedia.org/w/api.php'
    params = {
        'format': 'json',
        'action': 'query',
        'prop': 'extracts|pageprops',
        'exintro': 1,
        'explaintext': 1,
        'redirects': 1,
        'titles': title,
    }

    response = requests.get(endpoint, params=params, headers=headers, timeout=10)

    if not response.ok:
        return {'information': [], 'metadata': 'respond error'}

    data = response.json()
    page = next(iter(data['query']['pages'].values()))
    extract = page.get('extract', '')
    pageprops = page.get('pageprops', [])
    is_disambiguation = any('disambiguation' in str(cat).lower() for cat in pageprops)
    start_idx = 2 if is_disambiguation else 0
    info_list = [line for line in extract.split('\n')[start_idx:] if line]

    if not info_list:
        return {'information': [], 'metadata': 'no info'}

    # Happy path return — only set what's needed
    if is_disambiguation:
        metadata = 'multiple entries'
    else:
        info_list = info_list[:1]
        metadata = 'one entry'

    return {'information': info_list, 'metadata': metadata}

first_sentence = get_wikipedia_intro_with_category('Putin')
print(first_sentence)

{'information': ['Vladimir Vladimirovich Putin (born 7 October 1952) is a Russian politician and former intelligence officer who has served as President of Russia since 2012, having previously served from 2000 to 2008. Putin also served as Prime Minister of Russia from 1999 to 2000 and again from 2008 to 2012.'], 'metadata': 'one entry'}
